### **Pre procesamos las noticias que tenemos**
_Por Juan Carlos Rodríguez-Raga y Andrés Mauricio Toloza Cruz_

Este código hace un pre procesamiento básico de las noticias para poderr hacer analisis posteriores. 

##### **Primer paso: descargar e importar nuestras librerias**
La libreria que usaremos principalmente será NLTK, pero también necesitamos Pandas:

    ! pip install nltk
    ! pip install pandas

Tambien usaremos otras librerias pero estas ya vienen instaldas en Python.

In [39]:
# Importamos las librerias que usaremos

import pandas as pd 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer, RegexpTokenizer

nltk.download('stopwords')

import os
import string
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
os.chdir('C:/Users/andre/OneDrive - Universidad de los andes/C.1. Centro de Datos/Noticias del mes/mes-en-noticias-1')

In [43]:
# Importamos nuestras bases de dato

caracol = pd.read_csv('raw-data/Caracol_Enero2025.csv')
el_espectador = pd.read_csv('raw-data/ElEspectador_Enero2025.csv')
semana = pd.read_csv('raw-data/Semana_Enero2025.csv')
lasilla = pd.read_csv('raw-data/LaSillaVacia_Enero2025.csv')

In [44]:
# Pegamos las bases de datos verticalmente

df_raw = pd.concat([caracol, el_espectador, semana, lasilla], axis = 0)

In [45]:
# Eliminamos duplicados

df_raw = df_raw.drop_duplicates()

In [46]:
len(df_raw)

1105

In [47]:
df_raw.head(3)

,title,published_date,top_image,link,full_link,text
0,"Laura Sarabia, canciller designada, hace “un l...",2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/laura...,"Laura Sarabia, canciller designada, hace “un l..."
1,Nicolás Maduro le ofrece al presidente Gustavo...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/nicol...,"NA través de Instagram, el autoproclamado mand..."
2,Federico Gutiérrez a Gustavo Petro tras sancio...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/feder...,Sigue la tensión internacional y nacional por ...


In [48]:
conteo = df_raw['link'].value_counts()

conteo

link
www.semana.com             395
www.lasillavacia.com       354
www.elespectador.com       284
www.noticiascaracol.com     72
Name: count, dtype: int64

In [49]:
error_count = df_raw['title'].str.contains('Error').sum()

error_count

np.int64(1)

In [50]:
# Eliminamos las filas que contienen la palabra error

df_raw = df_raw[~df_raw['title'].str.contains('Error')]

In [51]:
len(df_raw)

1104

##### **Segundo paso: tokenizamos y limpiamos el texto**

En esta sección creamos una nueva columna con los artículos tokenizados y eliminamos las StopWords

In [52]:
stop_words = nltk.corpus.stopwords.words('spanish')

punc = list(set(string.punctuation))
punc.append('¿')
punc.append('¡')

In [53]:
# Cargamos nuestro tokenizador 

def casual_tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [54]:
# Hacemos un función para limpiar el texto

def process_text(text):
    text = casual_tokenizer(text)
    text = [each.lower() for each in text]
    text = [re.sub('[0-9]+', '', each) for each in text]
    text = [w for w in text if w not in punc]
    text = [w for w in text if w not in stop_words]
    text = [each for each in text if len(each) > 1]
    text = [each for each in text if ' ' not in each]
    return text

In [56]:
df_raw.loc[:, 'processed_text'] = df_raw['text'].apply(process_text)

In [63]:
df_raw.head(3)

,title,published_date,top_image,link,full_link,text,processed_text
0,"Laura Sarabia, canciller designada, hace “un l...",2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/laura...,"Laura Sarabia, canciller designada, hace “un l...","[laura, sarabia, canciller, designada, hace, l..."
1,Nicolás Maduro le ofrece al presidente Gustavo...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/nicol...,"NA través de Instagram, el autoproclamado mand...","[na, través, instagram, autoproclamado, mandat..."
2,Federico Gutiérrez a Gustavo Petro tras sancio...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/feder...,Sigue la tensión internacional y nacional por ...,"[sigue, tensión, internacional, nacional, rifi..."


In [66]:
# Guardamos nuestra base de datos

df_raw.to_csv('processed-data/NoticiasPre_Enero2025.csv', index = False)
df_raw.to_excel('processed-data/NoticiasPre_Enero2025.xlsx', index = False)